In [3]:
import json
%run "Papers.ipynb"

In [ ]:
def search_papers(leveled_search_terms, verbose=False):
    page = 1
    search_filter = ','.join([f'title.search:{search}' for search in searches])
    url = f"https://api.openalex.org/works?page={page}&filter={search_filter}"
    if verbose:
        print(f"URL: {url}")

    works = []
    data = requests.get(url).json()
    data = [] if 'results' not in data.keys() else data['results']
    
    while len(data) > 0:
        works += data
        page += 1
        url = f"https://api.openalex.org/works?page={page}&filter={search_filter}"
        if verbose:
            print(f"URL: {url}")
        data = requests.get(url).json()
        if 'results' in data.keys():
            data = data['results']
        else:
            break

    # Print Size and 1st 10 Titles
    if verbose:
        num_works = len(works)
        print(f"Number Works: {num_works}")
        for i in range(0, min(10, num_works)):
            print(works[i]["title"])

    # Create Dataset
    df = pd.DataFrame([], columns=columns)
    for work in works:
        work = pd.DataFrame([[work["title"], revert_abstract(work["abstract_inverted_index"]), work["doi"], 0]],  columns=columns)
        df = pd.concat([df, work])
    df.reset_index(drop=True, inplace=True)

    return df



In [6]:
with open('SearchTerms.json', 'r') as file:
    data = json.load(file)
print(data)

{'0': ['trait', 'phenotype'], '1': ['trait-mediated', 'higher-order', 'polymorphism', 'indirect', 'apparent', 'resource', 'keystone', 'intraguild', 'intransitive', 'trophic', 'competition', 'mutual', 'predation', 'chain', 'interaction', 'modification']}
